In [ ]:
#Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#SPDX-License-Identifier: MIT-0

In [ ]:
!pip install -q boto3==1.34.149

# [Optional] Clean up
Execute the below cells if you want to delete your OpenSearch collection and associated encryption, networking and data access policies


In the following code cell, previously saved variables (`collection_id`, `encryption_policy_name`, and `network_policy_name`) are being loaded from the notebook's storage using the `%store -r` magic command in Jupyter Notebook. These variables were likely defined and stored in earlier code cells or notebooks, and are now being retrieved for use in subsequent computations or operations within this notebook.

In [ ]:
#loading saved variables from previous notebooks
%store -r collection_id
%store -r encryption_policy_name
%store -r network_policy_name
%store -r data_access_policy_name
%store -r identity_arn
%store -r kb_id
%store -r kb_role_name

Or set those manually if you need.

In [ ]:
#You can also manually set them if needed

#collection_id = ""
#encryption_policy_name = ""
#network_policy_name = ""
#data_access_policy_name = ""
#identity_arn = ""
#kb_role_name = ""
#kb_id = ""

## Delete Amazon Bedrock KB related objects

In [ ]:
# Create a Boto3 client for Amazon Bedrock
bedrock_client = boto3.client('bedrock-agent')

try:

    # Delete the knowledge base
    bedrock_client.delete_knowledge_base(
        knowledgeBaseId=kb_id
    )
    print(f"Knowledge base {kb_id} deleted successfully")

except bedrock_client.exceptions.ResourceNotFoundException:
    print(f"Knowledge base {kb_id} not found")
except Exception as e:
    print(f"Error deleting knowledge base: {e}")


### Delete the underlying kb aim role

In [ ]:
iam_client = boto3.client("iam")

try:
    # List attached policies
    attached_policies = iam_client.list_attached_role_policies(
        RoleName=kb_role_name
    )["AttachedPolicies"]

    # Detach policies
    for policy in attached_policies:
        iam_client.detach_role_policy(
            RoleName=kb_role_name,
            PolicyArn=policy["PolicyArn"]
        )
        print(f"Detached policy {policy['PolicyArn']} from role {kb_role_name}")

    # List inline policies
    inline_policies = iam_client.list_role_policies(
        RoleName=kb_role_name
    )["PolicyNames"]

    # Delete inline policies
    for policy_name in inline_policies:
        iam_client.delete_role_policy(
            RoleName=kb_role_name,
            PolicyName=policy_name
        )
        print(f"Deleted inline policy {policy_name} from role {kb_role_name}")

    # Delete the role
    iam_client.delete_role(RoleName=kb_role_name)
    print(f"Deleted role {kb_role_name}")

except iam_client.exceptions.NoSuchEntityException:
    print(f"Role {kb_role_name} does not exist")
except Exception as e:
    print(f"Error deleting role: {e}")


In the following code cell, an instance of the AWS boto3 client for OpenSearch Serverless is created. This client allows you to interact with the OpenSearch Serverless service, which is a fully managed search and analytics service provided by AWS. The code then prints a message indicating that it will delete the specified collection and associated policies, such as encryption policy, network policy, and data access policy, based on the provided identifiers and names.

In [ ]:
import boto3

# Create OpenSearch Serverless client
os_client = boto3.client('opensearchserverless')

print(f"You are about to delete the following collection: {collection_id}\nand the below policies:\n{encryption_policy_name},\n{network_policy_name},\n{data_access_policy_name}")

In the following code cell, we will delete opensearch collection

In [ ]:
import random
try:
    # Delete the collection
    response = os_client.delete_collection(
        clientToken='unique-token' + str(random.randint(1000, 9999)),
        id=collection_id
    )
    print(f"Collection deleted: {response['deleteCollectionDetail']['id']}")
except Exception as e:
    print(f"Error deleting collection: {e}")

In the following code cell, we will delete network policy for the Opensearch collection.

In [ ]:
# Delete the network policy
try:
    response = os_client.delete_security_policy(
        name=network_policy_name,
        type='network'
    )
    print("Network policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Network policy not found")
except Exception as e:
    print(f"Error deleting network policy: {e}")

In the following code cell, we will delete encryption policy for the Opensearch collection

In [ ]:
# Delete the encryption policy
try:
    response = os_client.delete_security_policy(
        name=encryption_policy_name,
        type='encryption'
    )
    print("Encryption policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Encryption policy not found")
except Exception as e:
    print(f"Error deleting encryption policy: {e}")

In the following code cell, we will delete data access policies for the Opensearch collection

In [ ]:
# Delete the data access policy
try:
    response = os_client.delete_access_policy(
        name=data_access_policy_name,
        type='data'
    )
    print("Data access policy deleted")
except client.exceptions.ResourceNotFoundException:
    print("Data access policy not found")
except Exception as e:
    print(f"Error deleting data access policy: {e}")    


## Bedrock Prompt management clean up

In [ ]:
%store -r sentiment_categorisation_prompt_id
%store -r sentiment_categorisation_prompt_arn
%store -r summarisation_prompt_id
%store -r summarisation_prompt_arn
%store -r extraction_prompt_id
%store -r extraction_prompt_arn

In [ ]:
import boto3
bedrock_agent_client = boto3.client('bedrock-agent')
prompt_identifiers = [sentiment_categorisation_prompt_id, summarisation_prompt_id, extraction_prompt_id]

try:
    for prompt_identifier in prompt_identifiers:
        # Delete the prompt
        response = bedrock_agent_client.delete_prompt(
            promptIdentifier=prompt_identifier
        )
        print(f"Prompt '{prompt_identifier}' deleted successfully.")
except Exception as e:
    print(f"Error deleting prompt: {e}")